In [1]:
from threading import Thread
import sys
import cv2
from queue import Queue


In [2]:
from imutils.video import FileVideoStream
from imutils.video import FPS
import numpy as np
import argparse
import imutils
import time
import cv2
from datetime import datetime



In [3]:
class Counts:
    def __init__(self):
        self._start_time=None
        self._num_occurences=0
        
    def start(self):
        self._start_time=datetime.now()
        return self
        
    def increment(self):
        self._num_occurences+=1
        

        
    def countsPerSec(self):
        elapsed_time= (datetime.now()-self._start_time).total_seconds()
        if elapsed_time!=0:
            return self._num_occurences/elapsed_time
        else:
            return self._num_occurences/1


In [4]:
def putIterationsPerSec(frame, iterations_per_sec):
    cv2.putText(frame,"{:.0f} iterations/sec".format(iterations_per_sec),
               (10,450),cv2.FONT_HERSHEY_SIMPLEX,1.0,(255,255,255))
    return frame

In [5]:
def noThreading(source=0):
    cap=cv2.VideoCapture(source)
    cps=Counts().start()
    
    while True:
        (grabbed,frame)=cap.read()
        if not grabbed or cv2.waitKey(1)==ord("q"):
            break
        frame= putIterationsPerSec(frame,  cps.countsPerSec())
        cv2.imshow("Video",frame)
        cps.increment()
        
    cap.release()
    cv2.destroyAllWindows()
    
    

In [6]:
#noThreading('Facial Mask Predictor\Samples\masks_video.mp4')

In [7]:
from threading import Thread
import cv2

class VideoGet:
    def __init__(self,src=0):
        self.stream=cv2.VideoCapture(src)
        self.grabbed,self.frame= self.stream.read()

        self.stopped=False
    
    def start(self):
        Thread(target=self.get, args=()).start()
        return self
    
    def get(self):
        while not self.stopped:
            if not self.grabbed:
                self.stop()
                
            else:
                (self.grabbed, self.frame)=self.stream.read()
    
    def stop(self):
        self.stopped= True
                

In [8]:
class VideoShow:
    def __init__(self,frame=None):
        self.frame =frame
        self.stopped =False
        
    def start(self):
        Thread(target=self.show, args=()).start()
        return self
        
    def show(self):
        while not self.stopped:
            cv2.imshow("Video", self.frame)
            if cv2.waitKey(1)==ord("q"):
                self.stopped = True            
    def stop(self):
        self.stopped =True
        

In [9]:
def threadVideoGet(source=0):
    video_getter=VideoGet(source).start()
    cps=Counts().start()
    
    while True:
        if cv2.waitKey(1)==ord("q") or video_getter.stopped:
            video_getter.stop()
            break
        frame=video_getter.frame
        frame=putIterationsPerSec(frame,cps.countsPerSec())

        cv2.imshow("Video",frame)
        cps.increment()
    
    
    cv2.destroyAllWindows()
    
    

In [10]:
def threadVideoShow(source=0):
    cap=cv2.VideoCapture(source)
    grabbed,frame=cap.read()
    video_shower=VideoShow(frame).start()
    cps=Counts().start()

    while True:
        grabbed,frame= cap.read()
        if not grabbed or video_shower.stopped:
            video_shower.stop()
            break
        
        frame=putIterationsPerSec(frame,cps.countsPerSec())
        video_shower.frame=frame
        
        cps.increment()
    
    
    cv2.destroyAllWindows()
    
    

In [11]:
threadVideoGet("Facial Mask Predictor\Samples\masks_video.mp4")
threadVideoShow("Facial Mask Predictor\Samples\masks_video.mp4")

In [12]:
def threadBoth(source=0):
    video_getter=VideoGet(source).start()
    video_shower=VideoShow(video_getter.frame).start()
    
    cps=Counts().start()
    while True:
        if video_getter.stopped or video_shower.stopped:
            video_shower.stop()
            video_getter.stop()
            break
            
        frame= video_getter.frame
        frame= putIterationsPerSec(frame, cps.countsPerSec())
        video_shower.frame=frame
        
        cps.increment()
        
        
    cv2.destroyAllWindows()

In [13]:
threadBoth("Facial Mask Predictor\Samples\masks_video.mp4")